In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio, guardar_df_en_minio, generar_ruta_fecha
import os
from dotenv import load_dotenv

In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [11]:
def extract_word_content(word_document):
    """
    Extrae contenido específico de un archivo Word (.docx) ya abierto.

    Args:
        word_document (docx.Document): Documento Word abierto con python-docx.

    Returns:
        dict: Diccionario con la información extraída.
    """
    extracted_data = {
        "Numero de Informe": None,
        "Nombre del Paciente": None,
        "Identificacion": None,
        "Edad": None,
        "Telefono": None,
        "Fecha de Toma de Muestra": None,
        "Fecha de Ingreso": None,
        "Fecha de Informe": None,
        "Entidad": None,
        "EPS": None,
        "Servicio": None,
        "Muestra Remitida": None,
        "Descripcion Macroscopica": None,
        "Descripcion Microscopica": None,
        "Diagnostico": [],
        "Comentario": None
    }

    try:
        # Obtener todo el texto del documento Word
        full_text = "\n".join([paragraph.text for paragraph in word_document.paragraphs])
        cleaned_text = " ".join(full_text.split())
        
        # Extraer información utilizando búsquedas
        if "INFORME PATOLOGIA No." in cleaned_text:
            extracted_data["Numero de Informe"] = cleaned_text.split("INFORME PATOLOGIA No.")[1].split()[0].strip()
        if "Nombre:" in cleaned_text:
            extracted_data["Nombre del Paciente"] = cleaned_text.split("Nombre:")[1].split("Historia:")[0].strip()
        if "Historia:" in cleaned_text:
            extracted_data["Identificacion"] = cleaned_text.split("Historia:")[1].split("Edad:")[0].strip()
        if "Edad:" in cleaned_text:
            extracted_data["Edad"] = cleaned_text.split("Edad:")[1].split()[0].strip()
        if "Teléfono:" in cleaned_text:
            extracted_data["Telefono"] = cleaned_text.split("Teléfono:")[1].split()[0].strip()
        if "Fecha recibo:" in cleaned_text:
            extracted_data["Fecha de Toma de Muestra"] = cleaned_text.split("Fecha recibo:")[1].split("DESCRIPCION MACROSCOPICA")[0].strip()
        if "FECHA DE INGRESO:" in cleaned_text:
            extracted_data["Fecha de Ingreso"] = cleaned_text.split("FECHA DE INGRESO:")[1].split()[0].strip()
        if "FECHA:" in cleaned_text:
            extracted_data["Fecha de Informe"] = cleaned_text.split("FECHA:")[1].split("Nombre")[0].strip()
        if "Entidad:" in cleaned_text:
            extracted_data["Entidad"] = cleaned_text.split("Entidad:")[1].split("Medico Sol:")[0].strip()
        if "EPS:" in cleaned_text:
            extracted_data["EPS"] = cleaned_text.split("EPS:")[1].split("SERVICIO")[0].strip()
        if "SERVICIO:" in cleaned_text:
            extracted_data["Servicio"] = cleaned_text.split("SERVICIO:")[1].split()[0].strip()
        if "MUESTRA REMITIDA:" in cleaned_text:
            extracted_data["Muestra Remitida"] = cleaned_text.split("MUESTRA REMITIDA:")[1].split()[0].strip()
        if "DESCRIPCION MACROSCOPICA" in cleaned_text:
            extracted_data["Descripcion Macroscopica"] = cleaned_text.split("DESCRIPCION MACROSCOPICA")[1].split("DESCRIPCION MICROSCOPICA")[0].strip()
        if "DESCRIPCION MICROSCOPICA" in cleaned_text:
            extracted_data["Descripcion Microscopica"] = cleaned_text.split("DESCRIPCION MICROSCOPICA")[1].split("DIAGNÓSTICO:")[0].strip()
        if "DIAGNÓSTICO:" in cleaned_text:
            diagnostic_section = cleaned_text.split("DIAGNÓSTICO:")[1].strip().split("\n")
            extracted_data["Diagnostico"] = [line.strip() for line in diagnostic_section if line.strip()]
        if "COMENTARIO:" in cleaned_text:
            extracted_data["Comentario"] = cleaned_text.split("COMENTARIO:")[1].strip()

    except Exception as e:
        print(f"Error al extraer datos del documento Word: {e}")

    return extracted_data

In [12]:
import pandas as pd
import fitz  # PyMuPDFimport docx  # python-docx para archivos Word
import docx  # python-docx para archivos Word

def process_all_word_in_minio(minio_client, bucket_name, folder_path):
    """
    Procesa todos los archivos Word (.docx) en una ruta específica de un bucket MinIO.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde están los archivos
    folder_path : str
        Ruta dentro del bucket (ej: 'informes/2023/')
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los datos extraídos de todos los archivos Word
    """
    results = []
    
    try:
        # Asegurar que la ruta termine con '/'
        if not folder_path.endswith('/'):
            folder_path += '/'
        
        # Listar objetos en la ruta especificada
        objects = minio_client.list_objects(bucket_name, prefix=folder_path, recursive=True)
        
        for obj in objects:
            if obj.object_name.lower().endswith('.docx'):
                print(f"Procesando archivo: {obj.object_name}")
                word_data = None
                word_document = None
                
                try:
                    # Obtener el objeto Word
                    response = minio_client.get_object(bucket_name, obj.object_name)
                    word_data = BytesIO(response.read())
                    
                    # Abrir el documento Word
                    word_document = docx.Document(word_data)
                    
                    # Extraer contenido
                    content = extract_word_content(word_document)
                    
                    # Agregar los datos al resultado
                    results.append({
                        "Archivo": obj.object_name.split('/')[-1],
                        "Numero de Informe": content.get("Numero de Informe"),
                        "Nombre del Paciente": content.get("Nombre del Paciente"),
                        "Identificacion": content.get("Identificacion"),
                        "Edad": content.get("Edad"),
                        "Telefono": content.get("Telefono"),
                        "Fecha de Toma de Muestra": content.get("Fecha de Toma de Muestra"),
                        "Fecha de Ingreso": content.get("Fecha de Ingreso"),
                        "Fecha de Informe": content.get("Fecha de Informe"),
                        "Entidad": content.get("Entidad"),
                        "EPS": content.get("EPS"),
                        "Servicio": content.get("Servicio"),
                        "Muestra Remitida": content.get("Muestra Remitida"),
                        "Descripcion Macroscopica": content.get("Descripcion Macroscopica"),
                        "Descripcion Microscopica": content.get("Descripcion Microscopica"),
                        "Diagnostico": content.get("Diagnostico"),
                        "Comentario": content.get("Comentario"),
                    })
                    
                except Exception as e:
                    print(f"Error procesando {obj.object_name}: {e}")
                finally:
                    if 'response' in locals():
                        response.close()
                        response.release_conn()
    
    except S3Error as e:
        print(f"Error al acceder al bucket {bucket_name}: {e}")
        raise
    except Exception as e:
        print(f"Error inesperado: {e}")
        raise
    
    # Convertir resultados en un DataFrame
    return pd.DataFrame(results)

In [13]:
df_resultados = process_all_word_in_minio(
    minio_client=minio_client,
    bucket_name="yachay-landing",
    folder_path="Laboratorios/Laboratorio Fernando Sanzon/ENERO"
)

# Mostrar resultados
print(f"Se procesaron {len(df_resultados)} archivos WORD")
display(df_resultados.head())

Procesando archivo: Laboratorios/Laboratorio Fernando Sanzon/ENERO/R0036-1089242582 CMVA.docx
Procesando archivo: Laboratorios/Laboratorio Fernando Sanzon/ENERO/R032-59805655 CMVA.docx
Procesando archivo: Laboratorios/Laboratorio Fernando Sanzon/ENERO/R033-5285368SC.docx
Procesando archivo: Laboratorios/Laboratorio Fernando Sanzon/ENERO/R034-30711337SC.docx
Procesando archivo: Laboratorios/Laboratorio Fernando Sanzon/ENERO/R35 -59834885 SC.docx
Se procesaron 5 archivos WORD


,Archivo,Numero de Informe,Nombre del Paciente,Identificacion,Edad,Telefono,Fecha de Toma de Muestra,Fecha de Ingreso,Fecha de Informe,Entidad,EPS,Servicio,Muestra Remitida,Descripcion Macroscopica,Descripcion Microscopica,Diagnostico,Comentario
0,R0036-1089242582 CMVA.docx,24FS0036.,CRIOLLO CAICEDO BLANCA NIDIA.,1089242582.,36,None,Enero 10 de 2024,None,Enero 16 de 2024.,C.M. VALLE DE ATRIZ.,None,None,None,"En un frasco con formol, rotulado con datos de...",En el estudio microscópico se identifica vesíc...,[VESICULA BILIAR LESION COLECISTECTOMIA - COLE...,None
1,R032-59805655 CMVA.docx,24FS0032.,MARTINEZ MARIA LUZ.,59805655.,56,None,Enero 5 de 2024,None,Enero 12 de 2024.,C.M. VALLE DE ATRIZ.,None,None,None,"En un frasco con formol, rotulado con datos de...",En el estudio histológico se identifica a nive...,[PIE IZQUIERDO (PRIMER DEDO) HISTORIA DE DIABE...,None
2,R033-5285368SC.docx,24FS033.,FRANCISCO PARMENIO ROJAS ALVAREZ.,5285368.,69,None,Enero 9 de 2024.,None,Enero 11 de 2024.,Particular.,None,None,None,"En un frasco con formol, rotulado con datos de...",Fragmentos de lecho ulceroso con evidencia de ...,[ESTOMAGO: BIOPSIA: -ADENOCARCINOMA DE TIPO IN...,None
3,R034-30711337SC.docx,24FS034.,MARIA ANGELICA MINGAN.,30711337.,73,None,Enero 9 de 2024.,None,Enero 11 de 2024.,Particular.,None,None,None,"En un frasco con formol, rotulado con datos de...",Fragmentos de mucosa gástrica corporal y antra...,[ESTOMAGO: BIOPSIA: - ADENOCARCINOMA POBREMENT...,None
4,R35 -59834885 SC.docx,24FS35,AIDA CAROLINA MERA,59539385,47,None,ENERO 11 de 2024.,None,ENERO 20 de 2024.,PARTICULAR.,None,None,None,"A) En un frasco con formol, rotulado con datos...",None,"[A) ESTOMAGO (ANTRO), BIOPSIA - GASTRITIS CRON...",None


In [14]:
ruta_fecha = generar_ruta_fecha()   

In [15]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_resultados,
    bucket_name="yachay-bronze",
    ruta_destino=f"laboratorios/{ruta_fecha}/laboratorio Fernando Sanzon/Fernando Sanzon",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/laboratorios/2025/05/06/laboratorio Fernando Sanzon/Fernando Sanzon.csv
